# old way

In [1]:
import sqlite3
import pandas as pd
import requests
import json
import time

In [35]:
# comment out if already created
# create a database
from pathlib import Path
Path('api_database.db').touch()

In [2]:
# connect with empty db
conn = sqlite3.connect('api_database.db')
c = conn.cursor()

In [ ]:
# create table
c.execute('''DROP TABLE streaming''')
c.execute('''CREATE TABLE streaming (oplajob text, timestamp int, receive at int)''')
query0=''' select* from streaming'''
pd.read_sql(query0, conn)

In [ ]:
# truncate table
# c.execute('''DELETE FROM streaming''')

In [ ]:
query0=''' select* from streaming'''
pd.read_sql(query0, conn)

# new way

In [6]:
from sqlalchemy import create_engine
import pandas as pd

In [7]:
engine = create_engine('sqlite:///api_database.db')

In [8]:
engine.connect()
print(engine)

Engine(sqlite:///api_database.db)


In [ ]:
print(engine.table_names())

In [10]:
from sqlalchemy import text
with engine.connect() as connection:
    result = connection.execute(text("select * from streaming"))
    for row in result:
        print(row)

In [11]:
result.fetchall() # because no data in the table

[]

In [7]:
result = engine.execute('''select * from streaming''')
for row in result:
    print (row)

In [8]:
result.fetchall()  # because no data in the table

[]

In [160]:
sql_query = pd.read_sql_query('SELECT * FROM streaming', engine)

In [ ]:
df = pd.DataFrame(sql_query)
df

In [13]:
from sqlalchemy import text

with engine.connect() as connection:
    result = connection.execute(text("INSERT INTO streaming (a,b,c) values(1,2,3)"))

In [ ]:
sql_query = pd.read_sql_query('SELECT * FROM streaming', engine)
df = pd.DataFrame(sql_query)
df

# main.py

In [ ]:
from fastapi import FastAPI
import sqlite3
import pandas as pd
from fastapi.responses import JSONResponse
import time
from sqlalchemy import create_engine
from sqlalchemy import text

engine = create_engine('sqlite:///api_database.db')
engine.connect()


# conn = sqlite3.connect('api_database.db')
# c = conn.cursor()

app = FastAPI()

# @app.get('/get/{timestamp}')
# async def get_timestamp(timestamp:int):
#     query0= "select * from streaming where timestamp = " + str(timestamp)
#     return pd.read_sql(query0,conn)

# @app.post('/post/{timestamp}')
# async def post_timestamp(timestamp:int):
#     conn = sqlite3.connect('api_database.db')
#     c = conn.cursor()
#     query=f"INSERT INTO streaming (oplajob,timestamp,receive) values('OPLA-587391',999,999)"
#     c.execute(query)   
#     return 'Done!'




@app.get('/get-all/{timestamp}')
async def get_timestamp(timestamp:int):
    with engine.connect() as connection:
        sql_query = pd.read_sql_query(f'SELECT * FROM streaming where timestamp = {timestamp}', engine)
        df = pd.DataFrame(sql_query)
        return df.to_json()
    
    
@app.post('/post-all/{timestamp}')
async def post_timestamp(timestamp:int):
    with engine.connect() as connection:
        connection.execute(text(f"INSERT INTO streaming (oplajob,timestamp,receive) values('job-123',{timestamp},11111111111111)"))
        return f'timestamp {timestamp} inserted!!!'